<p style="float:right">
<img src="images/logos/cu.png" style="display:inline" />
<img src="images/logos/cires.png" style="display:inline" />
<img src="images/logos/nasa.png" style="display:inline" />
</p>

# Python, Jupyter & pandas: Module 4

## Using [xarray](http://xarray.pydata.org/en/stable/) and [pandas](http://pandas.pydata.org/) for analysis

In [1]:
%matplotlib inline
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt 

We can find a nice timeseries to examine.

David Robinson's Rutgers Northern hemisphere snowcover is a coarse (88 x 88)
northern hemisphere grid, with data going back to 1966.

http://climate.rutgers.edu/snowcover/docs.php?target=datareq

_Robinson, David A., Estilow, Thomas W., and NOAA CDR Program (2012):NOAA
Climate Date Record (CDR) of Northern Hemisphere (NH) Snow Cover Extent
(SCE), Version 1. [indicate subset used]. NOAA National Climatic Data
Center. doi:10.7289/V5N014G9 [access date]._

Following along the initial html link above we can find the opendap (DODS)
endpoint and access it via the netCDF4 python package.

In [0]:
import netCDF4
snowcover_url = 'http://www.ncdc.noaa.gov/thredds/dodsC/cdr/snowcover/nhsce_v01r01_19661004_latest.nc'

Open and connect the opendap endpoint.  

In [0]:
%%time 
ds = netCDF4.Dataset(snowcover_url)

Examine the netcdf attributes.

In [0]:
ds.ncattrs()

In [0]:
ds.cdr_variable

In [0]:
ds.title

Look at what variables are provided in the file.

In [0]:
ds.variables.keys()

attatch some metadata variables.

In [0]:
latitude = ds.variables['latitude']
longitude = ds.variables['longitude']
land = ds.variables['land']
area = ds.variables['area']

In [0]:
latitude

So we see it's an 88 x 88 grid of floats 

So what area does the grid cover?

In [0]:
from mpl_toolkits.basemap import Basemap
from ipywidgets import interact
import ipywidgets as widgets

@interact(longitude_0=widgets.IntSlider(min=-165,max=-15,step=30,value=-105))
def plot_land(longitude_0=-80):
    plt.figure(figsize=(10, 10))
    m = Basemap(projection='npstere', boundinglat=30, lon_0=longitude_0)
    m.drawcoastlines()
    m.pcolor(longitude[:], latitude[:], land[:], latlon=True, cmap='Accent')
    plt.draw()



In [0]:
%%time
snowcover = ds.variables['snow_cover_extent']

In [0]:
snowcover

we have attatched to a data set with 2574 88 x 88 grids where `1 = snow_covered` and `0 = no_snow`

This step copies all of the data from the url to your data variable.  It can take a long time. ~5min.

In [0]:
%%time
all_data = snowcover[:,:,:]

In [0]:
single_week = all_data[1000, : , :]  # just choose a snowy index

In [0]:
plt.imshow(single_week, interpolation='nearest')

In [0]:
plt.imshow(area, interpolation='nearest')
cb = plt.colorbar()
cb.set_label('Grid Cell Area: $km^2$')

use numpy's multiplication to multiply the cells to get a snowcovered area per cell.

In [0]:
plt.imshow(single_week * area[:], interpolation='nearest')

define a quick routine to compute the total snowcovered area for a grid.

In [0]:
def snowcover_area_km2(grid, area):
    return np.sum(grid * area)

compute each weekly total snow covered area in km^2

In [0]:
print(all_data.shape)

In [0]:
weeks = all_data.shape[0]
grid_area = area[:]
total_area = np.ma.zeros(weeks)
for i in np.arange(weeks):
    total_area[i] = snowcover_area_km2(all_data[i, :, :], grid_area)
    


read and convert the time data into datetime objects.

In [0]:
time = ds.variables['time']
times = netCDF4.num2date(time[:], time.units)

In [0]:
with mpl.rc_context(rc={'figure.figsize': (15,2)}):
    plt.plot(times, total_area)
    plt.title('Northern Hemisphere Weekly Snow Covered Area')

In [0]:
with mpl.rc_context(rc={'figure.figsize': (15,2)}):
    plt.plot(times[100:120], total_area[100:120], marker='.')
    plt.title('subset of weekly NH snowcover data')

We have weekly data at 7 day resolution, but we're interested in monthly averages.

In [0]:
offset=344
times[offset+1] - times[offset]

This is ok.  But Pandas provides lots of routines for working with timeseries data. Let's create a [pandas.Series](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.Series.html) timeseries `ts` from the data and times.

"[Series](http://pandas.pydata.org/pandas-docs/stable/dsintro.html#series) is a one-dimensional labeled array capable of holding any data type (integers, strings, floating point numbers, Python objects, etc.). The axis labels are collectively referred to as the index. The basic method to create a Series is to call:"

      s = pd.Series(data, index=index)



In [0]:
import pandas as pd    # by convention import pandas as pd

In [0]:
ts = pd.Series(total_area, index=times)

the `pd.Series.head()` function lets you examine a items from a series object

In [0]:
ts.head()

the `pd.Series.describe()` method give you a statistical overview of your series.

In [0]:
ts.describe()

The `pandas.Series` has a built in plot method that will give us something like our original data

In [0]:
with mpl.rc_context(rc={'figure.figsize': (15,2)}):
    ts.plot(title='Northern Hemisphere Snow Covered Area: $km^2$')

Examine the index of our `pd.Series`.

In [0]:
print(ts.index)

Pandas tells us it's a [DatetimeIndex](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.DatetimeIndex.html) and that link is to the documentation and it is pretty overwhelming.  Let's just look at a few things one can do with a DatetimeIndex.

Create a little subindex to play with. (subset by offset )

In [0]:
subindex = ts.index[50:70]

In [0]:
subindex

In [0]:
subindex.year

In [0]:
subindex.month

In [0]:
subindex.day

In [0]:
subindex.dayofyear

You can select data based on the index.

Select all data that fell in the month of December during any year.

In [0]:
ts[ts.index.month == 12].head(10)

select data from year 2000

In [0]:
ts[ts.index.year == 2000].head()


What was the maximum value of all data points?

In [0]:
ts.max()

On what day did that maxium occur?

In [0]:
ts[ts == ts.max()]

When did snow covered area reach at least 95% of the max ever?

In [0]:
ts[ts >= ts.max() * .95]

## resample timeseries 

Subsetting and selecting is useful. But in this case let's say we need to compare our total snowcovered area with some other monthly derived geophysical constant. We are going to have to turn our weekly data into monthly data.

We can use the `pd.Series.resample()` method to "align" our data with months.

In [0]:
ts.resample('MS').mean().head()

In [0]:
with mpl.rc_context(rc={'figure.figsize': (15,2)}):
    ts.resample('MS').mean().plot(title='NH Monthly Average Snow Covered Area')


In [0]:
with mpl.rc_context(rc={'figure.figsize': (15,2)}):
    ts[200:250].resample('MS').mean().plot(marker='x')
    ts[200:250].plot(marker='.')
    plt.title('50 Weeks of NH snowcover data')


In [0]:
with mpl.rc_context(rc={'figure.figsize': (15,2)}):
    ts.resample('MS').mean()[100:150].plot(title='50 months of snowcover data')


So what is happening when we call resample is to take the mean of all values that fell into a month.  We can show that explicitly.

original values for Nov 1966

In [0]:
ts.iloc[3:8]

Mean of Nov 1966 values

In [0]:
ts.iloc[4:8].mean()

and the resampled timeseries head

In [0]:
ts.resample('MS').mean().head(3)

you can see the '1966-11-01' values is the same as the mean of the data from
[1966-11-07, 1966-11-14, 1966-11-21, 1966-11-28]


But really in in this case, our files are "indexed" and  represents the start date of a 7 day dataset. Therefore, the data labeld 1966-10-31 represents the week beginning on that day through 11-06 and we should include those data values when we compute the November-1966 mean.

You could do a computation where you compute weights for each file based on how many days are in the target month and do weighted means, but with Pandas, there's an easier way

We can sample the data to a Daily period before sampling to a month period. We fill between the indexes with `ffill()`

Here's what the forward filled timeseries sampled to Days looks like around the beginning of the month.

In [0]:
ts.resample('D').ffill().iloc[18:35]

So we've filled every day with a value based on its input file period.

Now we can resample to monthly.

And you can see that the monthly values differe from the previous computation and are correctly weighted.

In [0]:
ts.resample('D').ffill().resample('MS').mean().head()

In [0]:
ts.resample('MS').mean().head()

So now we have a monthly timeseries of Total Northern Hemisphere Snow Cover.  What should we do with it?

In [0]:
monthly = ts.resample('D').ffill().resample('MS').mean()

In [0]:
with mpl.rc_context(rc={'figure.figsize': (15,3)}):
    monthly[monthly.index.month == 2].plot(linestyle='-', label='February', legend=True)
    monthly[monthly.index.month == 5].plot(marker='.', label='May', legend=True, title='Compare months?') 


We could answer the question
Which February has the lowest Snowcover?

In [0]:
monthly[monthly.index.month == 2].idxmin()

What are the rankings of snowcover for March from greatest to least?

In [0]:
march = monthly[monthly.index.month ==  3]
rank = march.rank(ascending=False)

In [0]:
rank.sort_values().index.year

Here would be a good point to introduce a [`pandas.DataFrame`](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.html)  

     Two-dimensional size-mutable, potentially heterogeneous tabular data
     structure with labeled axes (rows and columns). Arithmetic operations
     align on both row and column labels. Can be thought of as a dict-like
     container for Series objects. The primary pandas data structure.

A DataFrame will allow us to align rank and values in a single object.

Create a simple pd.DataFrame from our monthly data using at March and its rank.
We see that the indexes are the dame.  One has total snowcover area and the other has the rank.

In [0]:
march.head()

In [0]:
rank.head()

Now create a dataframe from a these series.

In [0]:
d = {'march': march, 'rank': rank}
df = pd.DataFrame(data=d)


In [0]:
df.head()

In [0]:
df.sort_values('rank', ascending=True).head()

Say we want to know anomalies from the mean for all march. (anomaly computation)

With a dataframe we just add another column where we have subtracted the mean of all Marches from each value.

In [0]:
df['march_anomaly'] = (df['march'] - df['march'].mean())

In [0]:
df.head()

Now we can create a dataframe from our original series and shape it how we like.

Create a new dataframe from `monthly` series

In [0]:
df = pd.DataFrame(monthly, columns=['snowcover'])
df.head()

If we want each month in its own column, we can set the index and then unstack months.

Set the index to Year and Month creating it as a multi-index first...

In [0]:
df = df.set_index([df.index.year, df.index.month])
df.head()

In [0]:
type(df.index)

In [0]:
print(df.index.levels[0])
print(df.index.levels[1])

You see that now the index is by [year and month] and is a [`pandas.MultiIndex`](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.MultiIndex.html)

Where in the index level 0 has years and level 1 has months.

Now use the unstack command to 'pivot' the data into rows of years and columns of months of snowcover.

In [0]:
year_by_month = df.unstack(level=1)
year_by_month.head()


If you wanted rows of months and columns of years, you would have unstacked level=0


In [0]:
month_by_year = df.unstack(level=0)
month_by_year.head(3)

In [0]:
month_by_year.columns

In [0]:
month_by_year['snowcover'][[1970,1980, 1990, 2000, 2010 ]]

In [0]:
month_by_year['snowcover'][[1970,1980, 1990, 2000, 2010 ]].plot()

I've heard there's a trend in the Northern Hemisphere June snowcover.

In [0]:
june_anomalies = year_by_month['snowcover'][6] - year_by_month['snowcover'][6].mean()
june_anomalies = june_anomalies.dropna()

In [0]:
with mpl.rc_context(rc={'figure.figsize': (15, 4)}):
    june_anomalies.plot(title='Northern Hemisphere Snow Cover Anomalies: June',kind='Bar', color='r')

    

Compute a least squares linear fit

In [0]:
slope, intercept = np.polyfit(june_anomalies.index.values, june_anomalies.values, 1)
p = np.poly1d([ slope, intercept])
best_fit = p(june_anomalies.index)

In [0]:
with mpl.rc_context(rc={'figure.figsize': (15, 4)}):
    june_anomalies.plot(title='Northern Hemisphere Snow Cover Anomalies: June',kind='Bar', color='r')
    plt.plot(best_fit, color='b', linestyle='--')
